In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
from fastai.imports import *
from fastai.torch_imports import *
from fastai.dataset import *
from fastai.learner import *
from fastai.metrics import *
from fastai.core import *
from fastai.model import *
from fastai.lm_rnn import *
from collections import *
from fastai.conv_learner import *
import matplotlib.pyplot as plt

In [3]:
#%env CUDA_LAUNCH_BLOCKING=1

In [4]:
preprocess = torchvision.transforms.Compose([
   torchvision.transforms.Resize(32),
   torchvision.transforms.CenterCrop(32),
   torchvision.transforms.ToTensor(),
])

In [5]:
class PasswordDataset(Dataset):
    def __init__(self, files, ys):
        self.files = files
        self.ys = ys
        
    def __getitem__(self, idx):
        #print(self.files[idx], self.ys[idx])
        out = (preprocess(PIL.Image.open(self.files[idx]))/255).float().cuda()[:3], self.ys[idx].long().cuda()
        return out
        #return torch.Tensor(np.asarray(PIL.Image.open(self.files[idx]))/255).float().cuda()[:, :, :3],X
        
    
    def __len__(self): return len(self.ys)
    
    @classmethod
    def from_files(cls, img_path, y_path, cv_idxs):
        files = np.array(sorted(glob(img_path), key=lambda x: int(re.findall('\d+', x)[0])))

        ys = None
        
        with open(y_path) as f:
            raw = f.read().split("\n")
            ys = np.array([int(x) for x in raw if x != ''])
        
        mask = np.zeros(len(files), dtype='bool')
        print(len(files))
        mask[cv_idxs] = True
        val_where = list(mask)
        trn_where = list(~mask)

        #trn_where = ~mask
        return cls(files[trn_where], torch.Tensor(ys[trn_where])), cls(files[val_where], torch.tensor(ys[val_where]))


In [6]:
image_paths = "/home/robert/Documents/typingviewer/data/vidya/image-*.jpg"

In [7]:
y_path = "/home/robert/Documents/typingviewer/data/y2.csv"

In [8]:
dataset_size = len(glob(image_paths))

In [9]:
cv_idxs = get_cv_idxs(dataset_size, val_pct=0.1)

In [10]:
cv_idxs

array([1424,  111, 1841,  968,  415, 1780,   70,  530,  617, 1033, 1394,  949, 1085,  128,  367, 1894, 1214,
        730,  374, 1329,  275,  861, 1189, 1273,  212,  777,  135, 1465, 1653, 1265,  654,  857, 1538, 1497,
        231, 1283, 1304, 1289,  593, 1735,  585,  324,  366,  555, 1550, 1690,  414, 1364,   99, 1383, 1228,
       1026, 1435,  237,  289, 1083, 1745,  382,  806, 1477, 1658, 1073,  838, 1870,  297, 1261, 1862, 1313,
        677, 1347, 1475, 1268, 1895,   65, 1058, 1393, 1672,  916,   29,  610, 1677,  707, 1298, 1774,  572,
       1592,  331, 1568, 1562,  485, 1414,  413, 1762, 1284,  453,  438,  964, 1029, 1667,  694,  322, 1350,
       1680,  196,  936,  596,  120, 1374, 1611,  759,  109,  305, 1829, 1454, 1843,  755,  344,  427,  561,
       1133,  845, 1623, 1839, 1594, 1317,  613,  979, 1124,  788,  710,  611, 1331,  514, 1164,  251,  976,
       1898,  480,  527,  239, 1606, 1582,  932,  254, 1688, 1222, 1055, 1734, 1567,  518, 1211, 1433, 1425,
        700,   23, 

In [11]:
trn_ds, val_ds = PasswordDataset.from_files(image_paths, y_path, cv_idxs)

1908


In [12]:
trn_ds[0]

(tensor(1.00000e-03 *
        [[[ 0.3537,  0.3537,  0.3537,  ...,  0.3537,  0.3537,  0.3537],
          [ 0.3537,  0.3537,  0.3537,  ...,  0.3537,  0.3537,  0.3537],
          [ 0.3537,  0.3537,  0.3537,  ...,  0.3537,  0.3537,  0.3537],
          ...,
          [ 0.3537,  0.3537,  0.3537,  ...,  0.3691,  0.3691,  0.3691],
          [ 0.3537,  0.3537,  0.3537,  ...,  0.3691,  0.3691,  0.3691],
          [ 0.3537,  0.3537,  0.3537,  ...,  0.3691,  0.3691,  0.3691]],
 
         [[ 0.3537,  0.3537,  0.3537,  ...,  0.3537,  0.3537,  0.3537],
          [ 0.3537,  0.3537,  0.3537,  ...,  0.3537,  0.3537,  0.3537],
          [ 0.3537,  0.3537,  0.3537,  ...,  0.3537,  0.3537,  0.3537],
          ...,
          [ 0.3537,  0.3537,  0.3537,  ...,  0.3691,  0.3691,  0.3691],
          [ 0.3537,  0.3537,  0.3537,  ...,  0.3691,  0.3691,  0.3691],
          [ 0.3537,  0.3537,  0.3537,  ...,  0.3691,  0.3691,  0.3691]],
 
         [[ 0.3537,  0.3537,  0.3537,  ...,  0.3537,  0.3537,  0.3537],
      

In [13]:
trn_dl = torch.utils.data.DataLoader(trn_ds, batch_size=4)
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=4)

In [14]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [15]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 11)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net().cuda()

In [16]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [17]:
next(iter(trainloader))[0].shape

torch.Size([4, 3, 32, 32])

In [18]:
next(iter(trn_dl))[0].shape

torch.Size([4, 3, 32, 32])

In [22]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trn_dl, 0):
        print(i)
        # get the inputs
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 5 == 4:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 5))
            running_loss = 0.0

print('Finished Training')

0
1
2
3
4
[1,     5] loss: 2.083
5
6
7
8
9
[1,    10] loss: 2.064
10
11
12
13
14
[1,    15] loss: 2.297
15
16
17
18
19
[1,    20] loss: 2.227
20
21
22
23
24
[1,    25] loss: 2.064
25
26
27
28
29
[1,    30] loss: 2.329
30
31
32
33
34
[1,    35] loss: 2.271
35
36
37
38
39
[1,    40] loss: 2.315
40
41
42
43
44
[1,    45] loss: 2.481
45
46
47
48
49
[1,    50] loss: 2.481
50
51
52
53
54
[1,    55] loss: 2.591
55
56
57
58
59
[1,    60] loss: 2.428
60
61
62
63
64
[1,    65] loss: 2.373
65
66
67
68
69
[1,    70] loss: 2.396
70
71
72
73
74
[1,    75] loss: 2.249
75
76
77
78
79
[1,    80] loss: 2.354
80
81
82
83
84
[1,    85] loss: 2.331
85
86
87
88
89
[1,    90] loss: 2.322
90
91
92
93
94
[1,    95] loss: 2.420
95
96
97
98
99
[1,   100] loss: 2.563
100
101
102
103
104
[1,   105] loss: 2.498
105
106
107
108
109
[1,   110] loss: 2.255
110
111
112
113
114
[1,   115] loss: 2.346
115
116
117
118
119
[1,   120] loss: 2.507
120
121
122
123
124
[1,   125] loss: 2.448
125
126
127
128
129
[1,   130] loss